In [51]:
import numpy as np
import pandas as pd
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from fuzzywuzzy import fuzz
import time

In [52]:
def convert_dis(distance):
    if (1 - (distance -1)* 0.1 > 0):
        return 1 - (distance -1)* 0.1
    else:
        return 0
    
def nlp_API(text, client):
    text = text
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    sentiment = client.analyze_sentiment(document=document).document_sentiment

    return sentiment.score

In [53]:
client = language.LanguageServiceClient()
all_races = np.load("all_races.npy", encoding="bytes")
fword = list(np.load("fword.npy",encoding="bytes"))
gender = ['men','woman','lesbian','gay','bisexual','transgender','Trans','queer','questioning','intersex']
# below are dict 
hate_All = np.load("hate_All.npy", encoding="bytes").item()
racial = np.load("racial.npy", encoding="bytes").item()
# this is a list of keys of hate_All
hate_keys = list(hate_All.keys())

In [54]:
file = open("data.txt", 'r')
sentences = []
label_true = []
count = 0
for line in file:
    if line.split("-")[0] == '3':
        label_true.append(1)
    else:
        label_true.append(0)
    sentences.append(line.split("-")[1].split())  

In [55]:
file1 = open("feature.txt", 'w')
count = 1
for sentence in sentences[14269:]:
    #print(count)
    sentence_changed = ' '.join(sentence)
    count += 1
    fword_index = []
    special_word_index = []
    feature_vec = []
    for i in range(len(sentence)):
        if sentence[i] in fword:
            fword_index.append(i)
        if sentence[i] in gender or sentence[i] in all_races or sentence[i] in racial.keys() or sentence[i] in hate_keys:
            special_word_index.append(i)
        # if no special word, return all zero vector
    if len(special_word_index) == 0:
        feature_vec = [0,0,0,0,0]
        #print(feature_vec)
        #break
    else:
        feature_vec.append(len(special_word_index))

        # eturne 2 distance between special word and fword
        min_dis = 100
        for fword_i in fword_index:
            for special_word_i in special_word_index:
                if(fword_i != special_word_i):
                    min_dis = min(min_dis, abs(fword_i - special_word_i))

        feature_vec.append(convert_dis(min_dis))

        # feture 3 offensiveness
        offensive = 0
        for special_word_i in special_word_index:
            if sentence[special_word_i] in hate_keys:
                offensive += float(hate_All[sentence[special_word_i]])
        feature_vec.append(offensive)

        # dict of racial and gender
        exist = 0
        for special_word_i in special_word_index:
            temp = sentence[special_word_i]
            if temp in racial.keys():
                for racial_value in racial[temp]:
                    racial_value = racial_value.lower()
                    if fuzz.partial_ratio(racial_value, sentence_changed) >= 0.88:
                        exist = 1
                        break
        feature_vec.append(exist)

        #print(special_word_index)
        # google api score
        api_value = 0
        for i in special_word_index:
            if i - 2 >= 0:
                left_bigram = sentence[i -2] + ' ' + sentence[i - 1] 
                try:
                    api_value = min(nlp_API(left_bigram, client), api_value)
                except:
                    time.sleep(1)
                    try:
                        api_value = min(nlp_API(left_bigram, client), api_value)
                    except:
                        time.sleep(1)
                        api_value = min(nlp_API(left_bigram, client), api_value)
            elif i - 1 == 0:
                left_bigram = sentence[i - 1] 
                try:
                    api_value = min(nlp_API(left_bigram, client), api_value)
                except:
                    time.sleep(1)
                    try:
                        api_value = min(nlp_API(left_bigram, client), api_value)
                    except:
                        time.sleep(1)
                        api_value = min(nlp_API(left_bigram, client), api_value)
            if i + 2 < len(sentence):
                right_bigram = sentence[i + 1] + ' ' + sentence[i + 2]
                try:
                    api_value = min(nlp_API(right_bigram, client), api_value)
                except:
                    time.sleep(1)
                    try:
                        api_value = min(nlp_API(right_bigram, client), api_value)
                    except:
                        time.sleep(1)
                        api_value = min(nlp_API(right_bigram, client), api_value)
            elif i + 1 < len(sentence):
                right_bigram = sentence[i + 1]
                try:
                    api_value = min(nlp_API(right_bigram, client), api_value)
                except:
                    time.sleep(1)
                    try:
                        api_value = min(nlp_API(right_bigram, client), api_value)
                    except:
                        time.sleep(1)
                        api_value = min(nlp_API(right_bigram, client), api_value)
        if api_value > 0:
            feature_vec.append(0)
        else:
            feature_vec.append(abs(api_value))

    #print(feature_vec)
    print(str(feature_vec[0])+ ' ' + str(feature_vec[1]) + ' ' + str(feature_vec[2]) + ' ' + str(feature_vec[3])+ ' ' + str(feature_vec[4]))

1 0 0.85 0 0
1 0 0.85 0 0.0
0 0 0 0 0
1 0 0.85 0 0
1 0 0.85 0 0.0
1 1.0 0.85 0 0.30000001192092896
1 0 0.85 0 0.0
0 0 0 0 0
1 0 0.85 0 0
0 0 0 0 0
1 0 0.85 0 0.0
1 0 0.85 0 0
1 0 0.85 0 0.0
1 0 0.85 0 0
0 0 0 0 0
3 0.8 2.18 0 0.0
0 0 0 0 0
0 0 0 0 0
1 0 0.85 0 0
0 0 0 0 0
0 0 0 0 0
1 0 0.85 0 0.30000001192092896
0 0 0 0 0
1 0 0.85 0 0.0
1 0 0.85 0 0.5
1 0 0.85 0 0.0
1 0 0.85 0 0.0
1 0.9 0.85 0 0.20000000298023224
1 0 0.85 0 0
1 1.0 0.85 0 0.4000000059604645
1 0 0.85 0 0
0 0 0 0 0
1 0 0.85 0 0.0
1 0 0.85 0 0.0
0 0 0 0 0
2 0.3999999999999999 0 0 0.800000011920929
1 0 0.88 0 0.0
1 0 0.88 0 0.0
1 0 0.88 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
1 0 0.88 0 0.10000000149011612
1 0 0.88 0 0.6000000238418579
1 0.9 0.88 0 0.30000001192092896
1 0 0.88 0 0.10000000149011612
1 0 0.88 0 0.0
0 0 0 0 0
1 0.8 0.88 0 0.0
1 0 0.88 0 0.0
1 0 0.88 0 0.0
0 0 0 0 0
0 0 0 0 0
1 0 0.88 0 0.0
1 0 0.88 0 0
1 1.0 0.88 0 0.20000000298023224
1 0 0.88 0 0.0
1 0 0.88 0 0
1 0 0.88 0 0.0
1 0 0.88 0 0
0 0 0

In [106]:
racial_value = "Texi drVer"
racial_value = racial_value.lower()
racial_value

'texi drver'

In [102]:
fuzz.partial_ratio("text driver", "indian stupid text driver")

100

In [112]:
for sentence in sentences[0:9]:
    print(sentence)

['warning:', 'penny', 'boards', 'will', 'make', 'you', 'a', 'faggot']
['fuck', 'dyke']
['at', 'least', 'i', 'dont', 'look', 'like', 'jefree', 'starr', 'faggot']
['is', 'a', 'fag"', 'jackie', 'jealous"', 'neeeee']
['you', 'heard', 'me', 'bitch', 'but', 'any', 'way', "i'm", 'back', 'th', 'texas', 'so', 'wtf', 'u', 'talking', 'about', 'bitch', 'ass', 'nigga']
['your', 'a', 'dirty', 'terrorist', 'and', 'your', 'religion', 'is', 'a', 'fucking', 'joke,', 'you', 'go', 'around', 'screaming', 'allah', 'akbar', 'doing', 'terrorist', 'shit.', 'dirty', 'faggot.']
['rt', 'looking', 'like', 'faggots?']
['well', 'i', 'thought', 'you', 'knew', 'actually', 'rt', 'man', 'why', "y'all", "didn't", 'tell', 'me', 'i', 'was', 'a', 'dick', 'riding', 'ass', 'faggot?', "y'all", 'not', 'real']
['i', 'know.', 'it', 'was', 'a', 'joke,', 'faggot.']
